In [ ]:
# Imports
import tensorflow.keras as K
import tensorflow as tf

In [4]:
# Install tensorflow v 2.9.1 so model.save() works
if ((tf.__version__) != '2.9.1'):
    %pip install tensorflow==2.9.1


In [5]:
# Data Preprocessing
def preprocess_data(X, Y):
    """
    X: numpy.ndarray of shape (m, 32, 32, 3) containing the CIFAR10 data
    Y: numpy.ndarray of shape (m,) containing the CIFAR10 data labels
    Returns X_p and Y_p
    """
    X_p = K.applications.efficientnet.preprocess_input(X, data_format=None)
    Y_p = K.utils.to_categorical(Y, 10)

    return X_p, Y_p

In [ ]:
# Prep variables for use in model building / training
MODEL_PATH = 'cifar10.h5'
optimizer = K.optimizers.Adam()
init = K.initializers.he_normal()

# Load CIFAR10 dataset
(x_train, y_train), (x_valid, y_valid) = K.datasets.cifar10.load_data()

# Pre-process the data
x_train_p, y_train_p = preprocess_data(x_train, y_train)
x_valid_p, y_valid_p = preprocess_data(x_valid, y_valid)

# input tensor
inputs = K.Input(shape=(32, 32, 3))

# Resize input
resize = K.layers.Lambda(
    lambda image: K.backend.resize_images(
    image, 240/32, 240/32, data_format='channels_last')
)(inputs)

In [ ]:
# Load pretrained EfficientNetB1 Base Bodel
base_model = K.applications.EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_tensor=resize,
    input_shape=(240, 240, 3)
)
base_model.trainable = False

# Add output layer
"""
Since we set include_top=False when we loaded our base model, our output shape
    is (None, 7, 7, 1280) in it's current state. We need to reduce this to a 
    10 row softmax to evaluate the baseline conditions of our model

K.layers.GlobalAveragePooling2D() operates on the height and width dimensions
    to return an array shape (None, 1280)

Last, we need to do the softmax to get shape (None, 10)

After we've determined baseline of our pretrained model, we'll add layers before
the GlobalAveragePooling2D layer to further train our model.
"""
out = base_model.output
out = K.layers.GlobalAveragePooling2D()(out)
out = K.layers.Dense(10, activation='softmax')(out)

# Compile Model
model = K.models.Model(inputs=inputs, outputs=out)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train Model
model.fit(x=x_train_p,
          y=y_train_p,
          batch_size=200,
          epochs=5,
          validation_data=(x_valid_p, y_valid_p))

In [ ]:
# Save Model
model.save('cifar10.h5')

In [8]:
# Load saved model and evaluate
load_model = K.models.load_model('cifar10.h5')
load_model.evaluate(x_valid_p, y_valid_p, verbose=1)

313/313 [==============================] - 286s 900ms/step - loss: 0.4941 - accuracy: 0.8334


[0.4941279888153076, 0.8334000110626221]